> ### Outline
- RNN / LSTM의 한계
- 트랜스포머의 핵심 원리
- Self - Attention
    - Query, Key, Value
    - Cross Attention vs Self Attention
    - Scaled Dot-Product Attention
- 트랜스포머의 전처리 과정
    - Positional Encoding
- 트랜스포머의 인코더/디코더 구조
    - Multi-Head Attention
        - Multi-Head Attention의 핵심 원리 및 장점
        - Masked Multi-Head Attention
    - Encoder-Decoder Attention

> ### 1. RNN / LSTM의 한계

1-1. RNN의 등장과 한계
- RNN의 필요성  
    기존의 머신러닝 기법으로는 순차(시퀀스) 데이터를 다루는 데 한계가 있음

- RNN의 핵심 구조  
    X1(입력층) -(U) h1(은닉층) -(W) O1(출력층)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|(V)  
    X2(입력층) -(U) h2(은닉층) -(W) O2(출력층)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|(V)   
    
    $h_(t-1)$의 정보가 $h_t$로 계속 전달되는 구조  
    V(순환 가중치)를 통해 과거 시점의 정보를 전달한다.

    '가중치 공유'  
    U(입력 가중치), W(출력 가중치), b(편향)을 반복적으로 적용(공유)  
    -> 학습 파라미터의 수 감소(효율적), 일반화 능력 향상

- RNN의 한계  
    1. 기울기 소실  
    기울기가 곱해지면서 0으로 수렴, 과거의 정보가 소실되는 현상  
    2. 기울기 폭주  
    기울기가 곱해지면서 무한대로 발산, 과거의 정보가 지나치게 왜곡되는 현상  
    3. 너무 느린 학습시간  
    순차적으로 학습하기 때문에 병렬처리가 불가능

    > -> 이러한 한계점들로 인해 '장기 의존성 문제'가 나타나게 됨.  
    이를 보완하는 대안(더 똑똑한 모델)으로 LSTM과 GRU가 등장!
    
1-2. LSTM의 등장과 한계
- LSTM의 핵심 구조 '3개의 게이트'  
    1. Forget Gate  
    과거의 정보 중 얼마나를 잊어버릴지 결정  
    2. Input Gate  
    현재 시점 정보 중 얼마나 셀 상태에 저장할지 결정  
    3. Output Gate  
    다음 층으로 어떤 정보를 전달할지 결정  

- LSTM의 한계
    - RNN보다 학습 파라미터가 많아지게 됨.  
    - Seq2Seq 모델은 LSTM과 다르게 입출력 일치 문제를 해결하였으나, '병목 현상' 문제가 있음  
    - 기울기 소실 문제가 완전히 해결되지 않음  
    - 병렬화 불가능 문제  

- 병목 현상  
    고정된 크기의 컨텍스트 벡터에, 긴 입력 시퀀스를 압축하려 할 시, 정보 손실이 발생  

> ### 2. Attention의 등장

LSTM, Seq2Seq 모델의 한계를 해결한 모델  
결과를 만들 때, 입력값의 모든 부분을 동일하게 보기 X  
대신 **가장 중요한 부분에 더 높은 가중치를 부여**하여 참고하는 아이디어

고정된 컨텍스트 벡터를 가진 Seq2Seq과 다르게, 출력의 시점마다 새로운 컨텍스트 벡터를 동적으로 계산할 수 있음 -> **병렬화가 가능**

- 어텐션의 과정  
    1. 입력을 벡터로 변환  
    2. 각 요소별 중요도 점수 산출  
        - 관련도 점수(Score): 단어들 사이의 관계와 유사성을 측정하여 매기는 점수  
        - 중요도 배분(Softmax): 관련도 점수를 0~1 사이의 값으로 변환, 모두 합치면 1이 되도록 조정 -> **어텐션 가중치**를 만듦  
        (* 소프트맥스 활성화 함수는 다중 클래스 분류 문제에서 각 클래스에 속할 확률을 추정하기 위해 주로 사용됨)  
    3. 중요도에 따른 입력 정보 조합  
        - 어텐션 가중치에 따라 특정 단어의 정보에 더 집중 + 낮은 중요도의 단어는 무시하면서 최종 결과를 도출

- 어텐션의 장점
    - 시간 경과에 대한 유연성: RNN에 비해 장기 의존성이 적고, 시퀀스의 거리에 따라 제약을 받지 않는다.  
    - 공간에 대한 유연성: (지역적인 부분을 참고하는) CNN에 비해 전역적인 부분까지 학습할 수 있다.  
    - **병렬화**: LSTM/GRU도 동일하게 가지고 있는 병렬 처리 불가 문제를 해결함

    > -> 주로 사용되는 NLP/LLM 분야 뿐만 아니라 이미지 관련 모델(디퓨전, 물체 감지, 이미지 분할)에도 사용됨

> ### 3. Self-Attention과 트랜스포머 구조의 등장  
(Attention is All You Need)

1. 논문 배경: RNN 기반 모델들의 한계점(느린 처리속도, 장기 의존성)  
-> 순차 계산 구조를 배제하고, 인코더 디코더 없이 어텐션만을 사용하는 아이디어

2. 트랜스포머 모델 제안  
RNN의 순환 구조 제거, 셀프 어텐션에 기반한 트랜스포머 모델  

3. 논문의 발견  
- Recurrence(RNN의 R!)이 필수가 아님을 증명: 데이처를 순차로 입력받지 않고, 동시에 입력받아 병렬 처리  
- Convolution(CNN의 C!)이 필수가 아님을 증명: '이웃' 정보만 보는 지역적(local) 방식이 아니라, 문장 전체를 보는 global한 방식    
- 병렬화 가능  
- 한꺼번에 입력받으면 순서 정보는?: 위치 정보를 계산하여 입력값에 더해주는 방식 채택(위치 인코딩)  
- 학습 속도 단축, 대량의 학습데이터 사용 가능  

> **-> 어텐션을 쓰던 딥러닝 모델들이 대부분 셀프 어텐션 방식을 채택하게 됨**  
BERT, GPT의 기반이 트랜스포머 모델이 됨  
NLP 외의 분야에도 확장

- 트랜스포머 구조: 어텐션만 사용하여 문장의 의미와 구조 파악

- 트랜스포머의 핵심 기술
    1. Self-Attention: 문장 안에서 한 단어가 다른 단어와 얼마나 중요한 관계인지 한번에 파악  
    2. Multi-Head Attention: 셀프 어텐션을 여러개의 머리로 동시에 실행  
    3. Positional Encoding: 단어의 위치정보를 인코딩하여, 동시 입력에서 위치 정보를 모르는 문제를 미리 방지  

> -> 즉, 인코더, 디코더 개념을 어텐션으로 대체한 것

> ### 3-1. 셀프 어텐션

(어텐션의 기본 구조)
- Query: '질문', '요청', 알고 싶거나 초점을 맞추고 있는 대상   
- Key: 검색의 대상이 되는 모든 정보들이 가지고 있는 '이름표'  
    쿼리와 키의 관련도를 측정  
- Value: 키와 묶여있는 실제 내용물.  
    관련도 계산이 끝나고 관련성이 높은 키가 선택되면, 그 키에 해당되는 밸류를 사용

### Cross-Attention(기존) vs Self-Attention

- 기존 Attention in Seq2Seq(Cross-Attention): 디코더가 인코더를 참조하는 과정  
    1. 쿼리 만들기  
    지금까지 생성된 단어들을 디코더가 보고, 지금 상태를 하나의 쿼리로 요약  
    2. 키, 밸류 불러오기  
    인코더가 입력 문장의 각 단어에 대한 벡터를 가지고 있음  
    3. 유사도 계산(어텐션 점수)  
    디코더의 Q - 인코더의 각각 K에 대한 유사도를 계산  
    -> Q와 K_i의 내적 점수를 Softmax로 정규화하고, 합산하면 1이 되는 가중치(알파)로 변환  
    4. Weighted Sum  
    인코더의 V들을 그 가중치로 섞어서 하나의 벡터로 만듦  
    5. 디코더가 다음 단어를 예측  
    디코더가 자신의 정보 + 4번의 컨텍스트 벡터를 합쳐서 다음에 올 단어를 예측

- Self-Attention: 문장 자체 안에서 내부의 관계를 파악  
    1. 단어의 프로필(벡터) 만들기(벡터 인베딩)  
    각 단어들을 고유한 특징을 담은 벡터로 변환  
    2. 단어 간 관계 점수 계산(정렬 점수)  
    내적을 통해 모든 단어 벡터끼리 얼마나 관련이 깊은지에 대한 '관계 점수'를 계산  
    3. 중요도 배분(Softmax & 어텐션 가중치)  
    소프트맥스 활성화 함수로 0~1 사이로 정규화,  
    총합이 1이 되는 어텐션 가중치로 변환  
    4. 새로운 프로필(벡터) 생성  
    어텐션 가중치를 바탕으로, 각 단어들은 다른 단어들의 정보를 자신에게 맞게 조합하여, **문맥이 반영된 새로운 프로필 벡터**가 됨.

    > -> "Attention is All You Need"가 의미하는 것  
    - Encoder에 Self-Attention 적용: RNN 없이도 입력 문장의 내부 문맥을 완벽하게 이해
    - Decoder에 Self-Attention 적용: RNN 없이도 지금까지 생성된 내부 문맥을 파악

    - 마지막으로 둘을 Cross-Attention으로 연결: 출력문을 생성하는 디코더가 입력문을 참고하도록 함  

### Scaled Dot-Product Attention  

- Dot-Product Attention: 쿼리와 키의 내적을 구해 유사도를 구하는 방법  
    - 문제점: 벡터의 차원이 커질수록 내적의 결과가 극단적(0으로 수렴하거나 너무 크게 발산하는)이 되는 문제가 있음. 기울기 소실 문제와 비슷  

    > -> 문제점을 해결하기 위해 고안된 Scaled 방법  
    내적을 진행한 후, 값의 크기를 맞추는 스케일링 과정 추가

    1. 내적을 계산하여 유사도 계산  
    2. 그 계산된 값을 $\sqrt{d_K}$ 로 나누는 스케일링 과정 **[추가된 부분]**  
    3. 소프트맥스 함수 적용하여 어텐션 가중치로 변환  
    4. 가중치를 밸류 벡터에 곱함


> ### 3-2. Multi-Head Attention

- Single-Head Attention  
**하나의** 가중치 행렬만을 학습하고 번역에 사용  
    - 512차원의 입력 벡터 -> 이 벡터의 어텐션 가중치 분포 계산 -> 새로운 512차원의 벡터 출력  
    - 문제점: **하나의** 어텐션 분포 안에서 각 단어 간 문법적인 관계, 의미적 관계, 위치 관계 등 모든 정보를 한꺼번에 담아서 가중 평균을 구함 -> 정작 중요한 관계를 놓칠 수 있음  

- Multi-Head Attention  
예) 512차원 입력 벡터 -> 64차원씩 8개로 나눔 -> 8개로 나눈 벡터에서 어텐션 스코어를 **병렬로 계산**하는 것  

- 작동 방식  
    1. 분할  
    큰 벡터를 여러 개의 작은 벡터 그룹(Q, K, V)으로 투영하여 여러 개의 관점으로 나눔  
    예) 입력: 512차원 벡터  
    이를 Query_1(64차원), Key_1(64차원), Value_1(64차원)... Query_8(64차원), Key_8(64차원), Value_8(64차원)으로 분할  
    2. 병렬 어텐션 계산  
    나누어진 n개의 헤드들이 병렬로 Scaled Dot-Product Attention을 계산  
    -> 헤드 i의 관점에서 문맥을 이해한 64차원 결과 벡터 Attention_Output_i 생성됨  
    3. 결합 및 최종 투영  
    n개의 헤드가 도출한 n개의 분석 결과를 합치는 과정  
        a. 결합: Concat을 통해 1~n의 아웃풋 벡터를 합침 -> 원본 입력벡터와 같은 사이즈가 됨  
        b. 최종 투영: a는 아직 각각의 아웃풋 벡터가 직렬로 붙여지기만 한 형태이므로, 이를 융합해 의미 있는 결론으로 만들어야 함.  
        또 다른 가중치 행렬을 곱해서$a * Wo$ = Final_Output 최종 벡터 완성!  


> ### 4. 트랜스포머의 전체 구조  

> ### 4-1. 트랜스포머의 전처리  

1. 토큰화  
    입력된 텍스트(문장, 단어)를 작은 토큰 단위로 조각내는 것  

2. 임베딩  
    1에서 토큰화된 각 토큰은 임베딩 단계에서 숫자들의 벡터로 변환됨  
    단어의 의미를 벡터 좌표 위에 나타내서, 유사한 단어는 벡터 공간 내에 비슷한 위치에 있게 하는 것  
    Word2Vec을 주로 이용하여 임베딩을 수행한다.

3. 순서 인코딩  
    트랜스포머는 (직렬로 처리하는 RNN과 다르게) 모든 단어를 병렬로 처리  
    -> 위치(순차) 정보를 주지 않으면 혼란이 생길 수 있음  
    각 단어의 임베딩 좌표에 '일관된 순서'를 따르는 다른 숫자를 추가 -> 순서 정보를 기록  
    -> 같은 단어라도 문장 내에서 위치가 다르면 다른 좌표(수정된 임베딩)을 가지게 됨

> ### 4-2. 트랜스포머의 인코더와 디코더  

![인코더디코더](https://wikidocs.net/images/page/159686/2_Encoders_Decoders.png)

1. 인코더  
입력 문장을 이해하고, 요약된 벡터로 변환  
토큰화, 임베딩, 순서 인코딩 + 인코더 레이어(Multi-Head Attention, Feed Forward Layer)로 구성  


    - 피드포워드 레이어란?  
    MLP(다층 퍼셉트론)이 2층으로 완전 연결된 구조  
    기능 a. 비선형성 추가: FFN 내부의 활성화 함수(예: ReLU)는 비선형성을 추가할 수 있음  
    기능 b. 표현력 증대 (차원 확장): FFN은 보통 입력 차원(예: 512)을 잠시 더 큰 차원(예: 2048)으로 확장시켰다가 다시 원래 차원(512)으로 줄이는 구조, 이렇게 하면 모델이 더 풍부한 특징을 학습할 수 있는 "작업 공간"을 잠시 제공하는 효과가 있음  

    - Add & Norm 이란?  
    기울기 소실 & 폭주 문제를 해결하기 위해, 출력에 입력값을 바로 전달해주는 것  
    -> 레이어가 쌓여도 기울기 소실과 폭주가 발생하지 않음  

    - 레이어 정규화  
    입력 벡터의 평균과 분산을 사용해 정규화  
    -> 학습 안정화, 모델이 더 빠르게 수렴  

2. 디코더  
인코더가 분석한 의미 벡터를 받아서, 출력 문장을 순차적으로 생성  
전처리 과정, **[디코더 레이어]**, 선형 레이어, 소프트맥스 레이어로 구성  

    **[여기서]** 인코더는 병렬 처리가 가능  
단, 디코더가 출력할 때는 순차적으로 생성하게 됨  
(예: 평소에는 LLM의 답변이 주루룩 출력되지만, 지피티 서버가 안좋을 때는 단어가 천천히, 순서대로 하나씩 출력되는 것을 관찰할 수 있습니다.)

    - **디코더 레이어**의 구성  
    1. **Masked** Multi-Head Attention  
    2. Encoder-Decoder Multi-Head Attention (Cross-Attention)  
    3. Feed Forward Layer  

    - Multi-Head 어텐션인데 왜 Masked 인가요?  
    미래 시점의 단어 정보를 참조하지 못하게 마스크를 적용  
    현재 단어보다 오른쪽(미래)에 있는 토큰의 어텐션 값을 0으로 만들어줌
    이는 '컨닝 방지용'으로, 과거의 단어들만 참조할 수 있게 하는 것  

    - Encoder-Decoder Multi-Head Attention  
    출력문을 생성하는 디코더가 출력(컨텍스트)와 입력문을 참고하도록 하여, 출력과 입력의 의미를 연결하는 과정  
    Q: 디코더의 이전 출력 내용을 바탕으로 만드는 '질문'  
    K: 인코더의 키들에서 관련성이 높은 키 확인  
    V: 관련성이 높은 키의 밸류에 높은 가중치(어텐션 점수)를 부여

> ### 4-3. 전체 흐름 정리  

[전처리]
1. 입력을 자름(토큰화)  
2. 각 토큰을 의미 벡터로 변환(임베딩)  
3. 단어들의 위치에 해당하는 벡터를 의미 벡터에 더하기(위치 인코딩)  

[인코더]  

4. 셀프 어텐션을 통해 단어가 그 문장 내의 다른 단어들과 어떤 관계인지 파악  
5. Q와 K의 내적을 통해 관계 점수를 구함, 소프트맥스로 정규화한 어텐션 가중치 생성  
6. 가중치를 V 벡터에 곱하고, 모두 더하여 새로운 벡터 생성  
7. 여러 개의 어텐션 헤드를 통해 다양한 관점의 문맥 파악(멀티헤드)  

[디코더]  

8. 시작 토큰으로 출력 시작.  
시작 토큰에서 생성된 Q가 인코더에게 "첫 단어에서 중요한 것을 질문", 그 중요한 단어(예: 주어)에 높은 어텐션  
9. 8의 출력과 시작토큰만 참고 하면서 문맥 생성  
새로운 Q가 인코더 출력에 "다음 단어에서 중요한 것을 질문"  
10. 9를 반복  
11. 종료 토큰 생성 -> 완료